In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ezc3d import c3d
from scipy.signal import resample
from scipy.io import loadmat, savemat
from joblib import load, dump
from tqdm import tqdm

In [2]:
PATH_DATASETS = '../../Dataset/'
PATH_DATASET_1 = 'gait-dbase-1/'

N_SAMPLES = 1024
FORCE_THRESHOLD = 20
TOE_HEIGTH_THRESHOLD = 40

In [3]:
path = os.path.join(PATH_DATASETS, PATH_DATASET_1)
subjects = os.listdir(path)

features = pd.DataFrame()
target = pd.DataFrame()

for subject in tqdm(subjects):
    files_path = os.path.join(path, subject)
    files = os.listdir(files_path)
    for filename in files:

        try:
            content = c3d(os.path.join(path, subject, filename), extract_forceplat_data=True)
            trajectory_x_l = content['data']['points'][1, 9:15, :]
            trajectory_y_l = content['data']['points'][1, 9:15, :]
            trajectory_z_l = content['data']['points'][2, 9:15, :]

            trajectory_x_r = content['data']['points'][1, 20:26, :]
            trajectory_y_r = content['data']['points'][1, 20:26, :]
            trajectory_z_r = content['data']['points'][2, 20:26, :]

            trajectory_x_l = resample(trajectory_x_l, N_SAMPLES, axis=1).T
            trajectory_y_l = resample(trajectory_y_l, N_SAMPLES, axis=1).T
            trajectory_z_l = resample(trajectory_z_l, N_SAMPLES, axis=1).T

            trajectory_x_r = resample(trajectory_x_r, N_SAMPLES, axis=1).T
            trajectory_y_r = resample(trajectory_y_r, N_SAMPLES, axis=1).T
            trajectory_z_r = resample(trajectory_z_r, N_SAMPLES, axis=1).T

            force_x_1 = content['data']['platform'][0]['force'][0, :]
            force_y_1 = content['data']['platform'][0]['force'][1, :]
            force_z_1 = content['data']['platform'][0]['force'][2, :]

            force_x_2 = content['data']['platform'][1]['force'][0, :]
            force_y_2 = content['data']['platform'][1]['force'][1, :]
            force_z_2 = content['data']['platform'][1]['force'][2, :]

            force_x_1 = resample(force_x_1, N_SAMPLES, axis=0).T
            force_y_1 = resample(force_y_1, N_SAMPLES, axis=0).T
            force_z_1 = resample(force_z_1, N_SAMPLES, axis=0).T

            force_x_2 = resample(force_x_2, N_SAMPLES, axis=0).T
            force_y_2 = resample(force_y_2, N_SAMPLES, axis=0).T
            force_z_2 = resample(force_z_2, N_SAMPLES, axis=0).T

            force_combined_z = force_z_1 + force_z_2
            force_mask = (force_combined_z >= FORCE_THRESHOLD)

            trajectory_x_l = trajectory_x_l[force_mask, :]
            trajectory_y_l = trajectory_y_l[force_mask, :]
            trajectory_z_l = trajectory_z_l[force_mask, :]

            trajectory_x_r = trajectory_x_r[force_mask, :]
            trajectory_y_r = trajectory_y_r[force_mask, :]
            trajectory_z_r = trajectory_z_r[force_mask, :]

            force_x_1 = force_x_1[force_mask]
            force_y_1 = force_y_1[force_mask]
            force_z_1 = force_z_1[force_mask]

            force_x_2 = force_x_2[force_mask]
            force_y_2 = force_y_2[force_mask]
            force_z_2 = force_z_2[force_mask]

            force_x_l = 0
            force_y_l = 0
            force_z_l = 0
            force_x_r = 0
            force_y_r = 0
            force_z_r = 0

            fm1_z_l = trajectory_z_l[:, 3]
            peak_height = np.max(fm1_z_l)
            force_mask_1 = (force_z_1 >= FORCE_THRESHOLD)
            area_of_interest = fm1_z_l[force_mask_1]

            if peak_height in area_of_interest:
                force_x_l = force_x_2
                force_x_r = force_x_1
                force_y_l = force_y_2
                force_y_r = force_y_1
                force_z_l = force_z_2
                force_z_r = force_z_1

            else:
                force_x_l = force_x_1
                force_x_r = force_x_2
                force_y_l = force_y_1
                force_y_r = force_y_2
                force_z_l = force_z_1
                force_z_r = force_z_2
            
            
            _features = pd.DataFrame(
                np.concatenate([trajectory_x_l, trajectory_y_l, trajectory_z_l], axis=1),
                columns=[
                    'lfal_x', 'ltam_x', 'lfcc_x', 'lfm1_x', 'lfm2_x', 'lfm5_x',
                    'lfal_y', 'ltam_y', 'lfcc_y', 'lfm1_y', 'lfm2_y', 'lfm5_y',
                    'lfal_z', 'ltam_z', 'lfcc_z', 'lfm1_z', 'lfm2_z', 'lfm5_z'
                ]
            )

            _target = pd.DataFrame(
                np.stack([force_x_r, force_y_r, force_z_r], axis=1),
                columns=['fx', 'fy', 'fz']
            )

            features = pd.concat([features, _features], axis=0)
            target = pd.concat([target, _target], axis=0)

        except Exception as e:
            # print('Error occured', e)
            pass


100%|██████████| 50/50 [03:25<00:00,  4.12s/it]


In [4]:
features

,lfal_x,ltam_x,lfcc_x,lfm1_x,lfm2_x,lfm5_x,lfal_y,ltam_y,lfcc_y,lfm1_y,lfm2_y,lfm5_y,lfal_z,ltam_z,lfcc_z,lfm1_z,lfm2_z,lfm5_z
0,315.805034,229.923447,268.002196,250.332418,307.221042,355.662087,315.805034,229.923447,268.002196,250.332418,307.221042,355.662087,97.109251,111.920361,70.391584,21.615834,35.859695,23.911881
1,316.201525,230.357598,268.320366,250.608644,307.473889,355.703976,316.201525,230.357598,268.320366,250.608644,307.473889,355.703976,97.858607,112.783901,71.370441,22.173299,36.328380,24.305099
2,316.585860,230.779727,268.625820,250.873300,307.716807,355.741374,316.585860,230.779727,268.625820,250.873300,307.716807,355.741374,98.628169,113.653814,72.376640,22.700847,36.773495,24.687503
3,316.858322,231.078397,268.846114,251.042544,307.873934,355.741809,316.858322,231.078397,268.846114,251.042544,307.873934,355.741809,99.459739,114.504124,73.484923,22.999202,37.035719,24.962563
4,316.963212,231.191474,268.940599,251.069639,307.902963,355.687135,316.963212,231.191474,268.940599,251.069639,307.902963,355.687135,100.376595,115.320214,74.736923,22.957537,37.026410,25.076662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,336.217297,430.378295,387.837740,455.164026,400.316828,347.992613,336.217297,430.378295,387.837740,455.164026,400.316828,347.992613,174.151094,161.404916,197.678529,38.806055,35.341668,35.137141
443,336.354757,430.507363,388.204196,455.454908,400.415580,348.475983,336.354757,430.507363,388.204196,455.454908,400.415580,348.475983,175.068587,162.012566,199.059009,39.585922,35.593265,35.497595
444,336.451750,430.588287,388.531698,455.670472,400.441345,348.910544,336.451750,430.588287,388.531698,455.670472,400.441345,348.910544,176.010172,162.651828,200.402243,40.481553,35.954112,35.948550
445,336.527553,430.640574,388.839039,455.832872,400.417946,349.308550,336.527553,430.640574,388.839039,455.832872,400.417946,349.308550,176.976744,163.322537,201.724798,41.460378,36.395788,36.475558


In [5]:
target

,fx,fy,fz
0,0.689221,2.345322,35.940089
1,1.209450,3.872579,63.547884
2,1.202230,3.615580,65.452605
3,1.350759,3.984159,77.536125
4,1.324120,3.898645,83.475254
...,...,...,...
442,2.130661,1.310097,52.028323
443,1.565818,1.001321,54.234873
444,1.029906,0.684647,49.369829
445,0.533591,0.396105,55.451614


In [6]:
path = '../../Dataset/Processed/data.joblib'
data = {
    'X': features,
    'y': target
}
dump(data, path)

['../../Dataset/Processed/data.joblib']